In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix
from itertools import product


# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/heartcsv/heart.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install gradio==3.43.1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 36.5 MB/s eta 0:00:0000:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.5 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=caa2345dfb4ef7175dd5f9a8a88a9f3552fea3823fc497d65eba33ef91986cf8
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0


In [3]:
import gradio as gr

In [4]:
df = pd.read_csv("/kaggle/input/hearts/heart.csv")

In [5]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [6]:
df['age'].min(), df['age'].max()

(29, 77)

In [7]:
df['sex'].min(), df['sex'].max()

(0, 1)

In [8]:
df['chol'].min(), df['chol'].max()

(126, 564)

In [9]:
X = df.drop(['target','cp', 'trestbps', 'fbs', 'thalach','exang', 'oldpeak', 'slope', 'thal', 'ca'], axis = 1).values  #train data  ['target','cp', 'trestbps', 'fbs', 'thalach','exang', 'oldpeak', 'slope', 'thal', 'ca']
Y = df['target']                           #test data


st = StandardScaler()
X = st.fit_transform(X)


X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.30, random_state = 101)

trainedlda = LinearDiscriminantAnalysis().fit(X_Train, Y_Train)
predictionlda = trainedlda.predict(X_Test)
print(confusion_matrix(Y_Test,predictionlda))
print(classification_report(Y_Test,predictionlda))


[[28 16]
 [13 34]]
              precision    recall  f1-score   support

           0       0.68      0.64      0.66        44
           1       0.68      0.72      0.70        47

    accuracy                           0.68        91
   macro avg       0.68      0.68      0.68        91
weighted avg       0.68      0.68      0.68        91



In [10]:
def inference(age, sex, ch, cardio):
    s = 0 if sex=='female' else 1
    df = pd.DataFrame([[age, s, ch, cardio]], 
                      columns=['Age', 'Sex', 'Cholestoral (in mg/dl)', 
                               'Resting electrocardiographic results'])
    df = st.transform(df)
    pred = trainedlda.predict_proba(df)[0]
    res = {'Kalp Hastalığı Yok': pred[0], 'Kalp Hastalığı Var': pred[1]}
    return res


sex = gr.inputs.Radio(['Kadın', 'Erkek'], label="CİNSİYET")
age = gr.inputs.Slider(minimum=1, maximum=100, default=22, label="Yaş")
ch = gr.inputs.Slider(minimum=120, maximum=560, default=200, label="Kolesterol (in mg/dl)")
cardio = gr.inputs.Radio([0, 1, 2], label="ELECTROCARDIOGRAPHIC Sonuçları")


#gr.Interface(inference, [age, sex, ch, cardio], "label", live=True).launch(share=True)
gr.Interface(fn=inference, inputs=[age, sex, ch, cardio], outputs="label").launch(share=True)


/tmp/ipykernel_33/3364272964.py:12: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  sex = gr.inputs.Radio(['Kadın', 'Erkek'], label="CİNSİYET")
/tmp/ipykernel_33/3364272964.py:12: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  sex = gr.inputs.Radio(['Kadın', 'Erkek'], label="CİNSİYET")
/tmp/ipykernel_33/3364272964.py:13: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  age = gr.inputs.Slider(minimum=1, maximum=100, default=22, label="Yaş")
/tmp/ipykernel_33/3364272964.py:13: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  age = gr.inputs.Slider(minimum=1, maximum=100, default=22, label="Yaş")
/tmp/ipykernel_33/3364272964.py:14: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and 

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://a831db50467915a63d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
